In [227]:
import networkx as nx
import pygame

In [6]:
# GLOBAL VARIABLES
WHITE = (255,255,255)
BLACK = (0,0,0)
GREY = (194,194,194)
BLUE = (25, 166, 194)
DARK_BLUE = (21, 134, 157)


In [219]:
def draw_contour(screen, size, color):
    pygame.draw.rect(screen, color, pygame.Rect(400-(size/2), 300-(size/2), size, size))
    
def draw_rounded_rect(screen, color, x, y, w, h, radius):
    pygame.draw.rect(screen, color, (x + radius, y, w - 2 * radius, h))
    pygame.draw.rect(screen, color, (x, y + radius, w, h - 2 * radius))
    pygame.draw.circle(screen, color, (x + radius, y + radius), radius)
    pygame.draw.circle(screen, color, (x + w - radius, y + radius), radius)
    pygame.draw.circle(screen, color, (x + radius, y + h - radius), radius)
    pygame.draw.circle(screen, color, (x + w - radius, y + h - radius), radius)
    
def draw_text(screen, text, x, y, font_size, color):
    font = pygame.font.Font(None, font_size)
    text = font.render(text, True, color)
    screen.blit(text, (x, y))
    
def draw_pieces(screen,positions):
    row = -1
    for i in range(9):
        if (i % 3 == 0): row += 1 
        if (positions[i] != 0):
            draw_rounded_rect(screen, BLACK, (1.485*(i % 3) + 1.0017)*124, (7.67*(row % 3) + 1.0017)*24, 184, 184, 10)
            draw_rounded_rect(screen, WHITE, (1.485*(i % 3) + 1.0017)*(124) + 2, (7.67*(row % 3) + 1.0017)*(24) + 2, 180, 180, 10) 
            draw_text(screen, str(positions[i]), (1.485*(i % 3) + 1.53)*124,(7.67*(row % 3) + 3)*24,150,BLACK)


In [229]:
class AStar8Puzzle:
    @staticmethod
    def find_blank(array_matriz):
        index= array_matriz.index(' ')
        return index
    
    @staticmethod
    def get_score_index(index_state, index_solucion):
        row_1 = [0, 1, 2]
        row_2 = [3, 4, 5]
        row_3 = [6, 7, 8]

        col_1 = [0, 3, 6]
        col_2 = [1, 4, 7]
        col_3 = [2, 5, 8]

        state_x = 0
        state_y = 0
        sol_x = 0
        sol_y = 0

        if index_solucion in row_1:
            sol_y = 0
        if index_solucion in row_2:
            sol_y = 1
        if index_solucion in row_3:
            sol_y = 2
        if index_state in row_1:
            state_y = 0
        if index_state in row_2:
            state_y = 1
        if index_state in row_3:
            state_y = 2
        if index_solucion in col_1:
            sol_x = 0
        if index_solucion in col_2:
            sol_x = 1
        if index_solucion in col_3:
            sol_x = 2
        if index_state in col_1:
            state_x = 0
        if index_state in col_2:
            state_x = 1
        if index_state in col_3:
            state_x = 2

        return abs(sol_x - state_x) + abs(sol_y - state_y)
    
    @staticmethod
    def get_score(lista_1nodo, lista_goal):
        score=0
        for i in range(len(lista_1nodo)):
            if i==0:
                nodo_index=lista_1nodo.index(' ')
                goal_index=lista_goal.index(' ')
            else:
                nodo_index=lista_1nodo.index(i)
                goal_index=lista_goal.index(i)
            score= score + AStar8Puzzle.get_score_index(nodo_index,goal_index)
            
        return score
    
    @staticmethod
    def get_n_sucesores(array_matriz):
        acciones = []
        index = AStar8Puzzle.find_blank(array_matriz)
        if index  in [0,1,2,3,4,5]:
            acciones.append('down')
        if index in [0,3,6,1,4,7]:
            acciones.append('right')
        if index in [2,5,8,1,4,7]:
            acciones.append('left')
        if index in [6,7,8,3,4,5]:
            acciones.append('up')

        return acciones
    
    @staticmethod
    def generate_n_sucesores(lista_acciones, nodo_anterior):
        index=AStar8Puzzle.find_blank(nodo_anterior)
        lista_n_sucesores=[]
        while lista_acciones:
            aux=-1
            lista_p=nodo_anterior.copy()
            accion=lista_acciones.pop()
            if accion=="up":
                aux=lista_p[index]
                lista_p[index]=lista_p[index-3]
                lista_p[index-3]=aux
            if accion=="down":
                aux=lista_p[index]
                lista_p[index]=lista_p[index+3]
                lista_p[index+3]=aux
            if accion== "right":
                aux=lista_p[index]
                lista_p[index]=lista_p[index+1]
                lista_p[index+1]=aux
            if accion=="left":
                aux=lista_p[index]
                lista_p[index]=lista_p[index-1]
                lista_p[index-1]=aux

            lista_n_sucesores.append(lista_p)

        return lista_n_sucesores
    
    @staticmethod
    def peso_heuristica(tupla_de_lista_tuplas):
        return tupla_de_lista_tuplas[2]#+len(tupla_de_lista_tuplas[1])
    
    @staticmethod
    def get_min_heuristic(lista_nodos_posible):
        nodo_mejor_heuristica=min(lista_nodos_posible, key=AStar8Puzzle.peso_heuristica)
        return nodo_mejor_heuristica
    
    @staticmethod
    def formato_comp_tuplas(nodo_actual, path_p, goal_node, reached):
        acciones_posibles=AStar8Puzzle.get_n_sucesores(nodo_actual)
        lista_sucesores=AStar8Puzzle.generate_n_sucesores(acciones_posibles, nodo_actual)
        sucesores_tuplas=[]
        for node in lista_sucesores:
            node_tupla=tuple(node)

            if node_tupla not in reached:
                path_nuevo=path_p+[node]
                score_nueva= AStar8Puzzle.get_score(node,goal_node)
                sucesores_tuplas.append((node,path_nuevo,score_nueva))
                reached.add(node_tupla)

        return sucesores_tuplas
    
    @staticmethod
    def imprimir_matriz(array_m):
        print("Matriz :")
        for i in range(0, len(array_m), 3):
            print(' '.join(map(str, array_m[i:i+3])))
        ####print("Fin de Matriz")
        print('-' * 20)
    
    @staticmethod
    def solve(scramble, solution):
        print(f'matriz_inicial:   {scramble}')
        reached=set()
        fila_a_star=[(scramble,[scramble],AStar8Puzzle.get_score(scramble,solution))]
        i=0
        while fila_a_star:
            i=i+1
            print(i)

            zip_nodo=AStar8Puzzle.get_min_heuristic(fila_a_star)
            #print (zip_nodo)

            fila_a_star.remove(zip_nodo)
            nodo_p, path_p, score_p = zip_nodo
            AStar8Puzzle.imprimir_matriz(nodo_p)

            if nodo_p==solution:
                return nodo_p, path_p, score_p

            tuplas_sucesores=AStar8Puzzle.formato_comp_tuplas(nodo_p,path_p,solution,reached)
            fila_a_star= fila_a_star+tuplas_sucesores
    

In [230]:
scramble = [2,8,3,1,' ',5,4,7,6]
solution = [1,2,3,4,5,6,7,8,' ']
AStar8Puzzle.solve(scramble, solution)

matriz_inicial:   [2, 8, 3, 1, ' ', 5, 4, 7, 6]
1
Matriz :
2 8 3
1   5
4 7 6
--------------------
2
Matriz :
2 8 3
1 5  
4 7 6
--------------------
3
Matriz :
2 8 3
1 5 6
4 7  
--------------------
4
Matriz :
2 8 3
1 5 6
4   7
--------------------
5
Matriz :
2   3
1 8 5
4 7 6
--------------------
6
Matriz :
2 8 3
1 7 5
4   6
--------------------
7
Matriz :
2 8  
1 5 3
4 7 6
--------------------
8
Matriz :
2 8 3
1   5
4 7 6
--------------------
9
Matriz :
2 8 3
1   6
4 5 7
--------------------
10
Matriz :
2 8 3
1 5 6
  4 7
--------------------
11
Matriz :
  2 3
1 8 5
4 7 6
--------------------
12
Matriz :
1 2 3
  8 5
4 7 6
--------------------
13
Matriz :
1 2 3
4 8 5
  7 6
--------------------
14
Matriz :
1 2 3
4 8 5
7   6
--------------------
15
Matriz :
1 2 3
4   5
7 8 6
--------------------
16
Matriz :
1 2 3
4 5  
7 8 6
--------------------
17
Matriz :
1 2 3
4 5 6
7 8  
--------------------


([1, 2, 3, 4, 5, 6, 7, 8, ' '],
 [[2, 8, 3, 1, ' ', 5, 4, 7, 6],
  [2, ' ', 3, 1, 8, 5, 4, 7, 6],
  [' ', 2, 3, 1, 8, 5, 4, 7, 6],
  [1, 2, 3, ' ', 8, 5, 4, 7, 6],
  [1, 2, 3, 4, 8, 5, ' ', 7, 6],
  [1, 2, 3, 4, 8, 5, 7, ' ', 6],
  [1, 2, 3, 4, ' ', 5, 7, 8, 6],
  [1, 2, 3, 4, 5, ' ', 7, 8, 6],
  [1, 2, 3, 4, 5, 6, 7, 8, ' ']],
 0)

In [225]:
values = (0,1,2,3,4,5,6,7,8)

# Initialize Pygame
pygame.init()

# Set the dimensions of the window
screen_width = 800
screen_height = 600

# Create the screen
screen = pygame.display.set_mode((screen_width, screen_height))

# Set the title of the window
pygame.display.set_caption("8-Puzzle")

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Fill the screen with a color (RGB: Red, Green, Blue)
    screen.fill(WHITE) 
    draw_contour(screen, 600, DARK_BLUE)
    draw_contour(screen, 585, BLUE)
    draw_contour(screen, 552, GREY)
    draw_pieces(screen, values)

    # Update the display
    pygame.display.flip()

# Quit Pygame
pygame.quit()

In [2]:
numbers = [1,2,3,
           4,5,6,
           7,8,9]



Hello World
